In [1]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal &> /dev/null
# Install rtree - Geopandas requirment
!apt install python3-rtree &> /dev/null
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git &> /dev/null
# Install descartes - Geopandas requirment
!pip install descartes &> /dev/null
!pip install plotly -U &> /dev/null

In [2]:
import folium
import geopandas as gpd
import pandas as pd
import shapely
import plotly.express as px

## 先ほど作成したデータをもとに営業地図を作成する

In [3]:
data = gpd.read_file('/content/drive/MyDrive/work/pycon_mini_shizu/data/data/shizuoka_merged_data.geojson')
data.head()

,KEY_CODE,AREA,JINKO,HYOSYO,CITYNAME,NAME,HTKSYORI,HTKSAKI,GASSAN,人口総数,男,女,世帯総数,総数、年齢「不詳」含む,総数０～４歳,総数５～９歳,総数１０～１４歳,総数１５～１９歳,総数２０～２４歳,総数２５～２９歳,総数３０～３４歳,総数３５～３９歳,総数４０～４４歳,総数４５～４９歳,総数５０～５４歳,総数５５～５９歳,総数６０～６４歳,総数６５～６９歳,総数７０～７４歳,総数１５歳未満,総数１５～６４歳,総数６５歳以上,総数７５歳以上,男の総数、年齢「不詳」含む,男０～４歳,男５～９歳,男１０～１４歳,男１５～１９歳,男２０～２４歳,男２５～２９歳,...,Ｅ 製造業,Ｆ 電気・ガス・熱供給・水道業,Ｇ 情報通信業,Ｈ 運輸業、郵便業,Ｉ 卸売業、小売業,Ｊ 金融業、保険業,Ｋ 不動産業、物品賃貸業,Ｌ 学術研究、専門・技術サービス業,Ｍ 宿泊業、飲食サービス業,Ｎ 生活関連サービス業、娯楽業,Ｏ 教育、学習支援業,Ｐ 医療、福祉,Ｑ 複合サービス事業,Ｒ サービス業（他に分類されないもの）,Ｓ 公務（他に分類されるものを除く）,Ｔ 分類不能の産業,雇用者（役員を含む）,自営業主（家庭内職者を含む）,家族従業者,総数_y,Ａ 管理的職業従事者,Ｂ 専門的・技術的職業従事者,Ｃ 事務従事者,Ｄ 販売従事者,Ｅ サービス職業従事者,Ｆ 保安職業従事者,Ｇ 農林漁業従事者,Ｈ 生産工程従事者,Ｉ 輸送・機械運転従事者,Ｊ 建設・採掘従事者,Ｋ 運搬・清掃・包装等従事者,Ｌ 分類不能の職業,総数,農林漁業就業者世帯,農林漁業・非農林漁業就業者混合世帯,非農林漁業就業者世帯,非就業者世帯,分類不能の世帯,WHOLE_NAME,geometry
0,221010010,175736.710,180,2.0,静岡市葵区,追手町,0.0,None,None,180.0,68.0,112.0,111.0,180.0,2.0,5.0,3.0,3.0,3.0,3.0,7.0,13.0,10.0,4.0,12.0,9.0,18.0,27.0,22.0,10.0,82.0,88.0,39.0,68.0,0.0,1.0,3.0,2.0,0.0,1.0,...,1.0,1.0,5.0,0.0,13.0,3.0,6.0,12.0,8.0,7.0,6.0,11.0,0.0,7.0,2.0,3.0,62.0,17.0,4.0,85.0,1.0,19.0,18.0,15.0,22.0,1.0,0.0,2.0,0.0,0.0,4.0,3.0,107.0,0.0,0.0,61.0,44.0,2.0,静岡市葵区追手町,"POLYGON ((138.38006 34.97625, 138.37999 34.976..."
1,221010020,62198.313,89,2.0,静岡市葵区,城内町,0.0,None,None,89.0,39.0,50.0,40.0,89.0,3.0,5.0,3.0,2.0,2.0,2.0,3.0,5.0,1.0,6.0,8.0,5.0,9.0,8.0,6.0,11.0,43.0,35.0,21.0,39.0,2.0,3.0,0.0,1.0,2.0,1.0,...,3.0,0.0,1.0,0.0,9.0,6.0,6.0,0.0,2.0,2.0,3.0,3.0,0.0,1.0,1.0,0.0,36.0,5.0,0.0,41.0,1.0,7.0,9.0,12.0,3.0,0.0,0.0,2.0,1.0,3.0,3.0,0.0,40.0,0.0,0.0,24.0,16.0,0.0,静岡市葵区城内町,"POLYGON ((138.38196 34.98145, 138.38143 34.981..."
2,221010030,143355.593,142,2.0,静岡市葵区,駿府町,0.0,None,None,142.0,54.0,88.0,67.0,142.0,6.0,5.0,6.0,4.0,3.0,1.0,3.0,8.0,14.0,9.0,10.0,11.0,8.0,14.0,12.0,17.0,71.0,51.0,25.0,54.0,1.0,3.0,1.0,1.0,2.0,1.0,...,2.0,0.0,1.0,1.0,29.0,1.0,2.0,6.0,14.0,5.0,3.0,2.0,0.0,3.0,1.0,6.0,38.0,25.0,12.0,79.0,0.0,6.0,8.0,26.0,21.0,0.0,0.0,6.0,0.0,2.0,4.0,6.0,67.0,0.0,0.0,40.0,23.0,4.0,静岡市葵区駿府町,"POLYGON ((138.38727 34.97837, 138.38724 34.978..."
3,221010040,190501.969,0,2.0,静岡市葵区,駿府公園,0.0,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,静岡市葵区駿府公園,"POLYGON ((138.38404 34.98125, 138.38562 34.980..."
4,221010050,165730.625,1209,2.0,静岡市葵区,西草深町,0.0,None,None,1209.0,573.0,636.0,523.0,1209.0,34.0,48.0,41.0,51.0,36.0,43.0,56.0,64.0,97.0,78.0,74.0,93.0,91.0,107.0,101.0,123.0,683.0,394.0,186.0,573.0,17.0,27.0,24.0,26.0,19.0,23.0,...,59.0,4.0,24.0,13.0,109.0,21.0,30.0,39.0,37.0,21.0,25.0,77.0,4.0,35.0,27.0,35.0,443.0,84.0,37.0,593.0,39.0,142.0,126.0,96.0,59.0,4.0,4.0,40.0,8.0,18.0,22.0,35.0,523.0,1.0,2.0,343.0,155.0,22.0,静岡市葵区西草深町,"POLYGON ((138.37944 34.97951, 138.37932 34.979..."


### 存在するデータ属性を確認し、役立ちそうなものをピックアップする


In [4]:
for col in data.columns[9:]:
    print(col)

人口総数
　男
　女
世帯総数
総数、年齢「不詳」含む
　総数０～４歳
　総数５～９歳
　総数１０～１４歳
　総数１５～１９歳
　総数２０～２４歳
　総数２５～２９歳
　総数３０～３４歳
　総数３５～３９歳
　総数４０～４４歳
　総数４５～４９歳
　総数５０～５４歳
　総数５５～５９歳
　総数６０～６４歳
　総数６５～６９歳
　総数７０～７４歳
　総数１５歳未満
　総数１５～６４歳
　総数６５歳以上
　総数７５歳以上
男の総数、年齢「不詳」含む
　男０～４歳
　男５～９歳
　男１０～１４歳
　男１５～１９歳
　男２０～２４歳
　男２５～２９歳
　男３０～３４歳
　男３５～３９歳
　男４０～４４歳
　男４５～４９歳
　男５０～５４歳
　男５５～５９歳
　男６０～６４歳
　男６５～６９歳
　男７０～７４歳
　男１５歳未満
　男１５～６４歳
　男６５歳以上
　男７５歳以上
女の総数、年齢「不詳」含む
　女０～４歳
　女５～９歳
　女１０～１４歳
　女１５～１９歳
　女２０～２４歳
　女２５～２９歳
　女３０～３４歳
　女３５～３９歳
　女４０～４４歳
　女４５～４９歳
　女５０～５４歳
　女５５～５９歳
　女６０～６４歳
　女６５～６９歳
　女７０～７４歳
　女１５歳未満
　女１５～６４歳
　女６５歳以上
　女７５歳以上
一般世帯数（世帯人員６人以上含む）
　世帯人員１人
　世帯人員２人
　世帯人員３人
　世帯人員４人
　世帯人員５人
一般世帯人員
１世帯当り人員
一般世帯総数
　親族世帯
　核家族世帯
　　うち夫婦のみの世帯
　　うち夫婦と子供から成る世帯
　核家族以外の世帯
　６歳未満世帯員のいる一般世帯総数
　１８歳未満世帯員のいる一般世帯総数
　６５歳以上世帯員のいる一般世帯総数
住宅に住む一般世帯
持ち家
民営借家
主世帯数
一戸建
長屋建
共同住宅
共同住宅１・２階建
共同住宅３～５階建
共同住宅６～１０階建
共同住宅１１階建以上
その他
総数_x
Ａ　農業、林業
　うち農業
Ｂ　漁業
Ｃ　鉱業、採石業、砂利採取業
Ｄ　建設業
Ｅ　製造業
Ｆ　電気・ガス・熱供給・水道業
Ｇ　情報通信業
Ｈ　運輸業、郵便業
Ｉ　卸売業、小売業
Ｊ　金融業、保険業
Ｋ　不動産業、物品賃貸業
Ｌ　学術研究、専門・技術サービス業
Ｍ　宿泊業、飲食サービス業
Ｎ　生活関連サービス業、娯

## 役立ちそうな属性をピックアップ

- 人口
- 子供が多い
    - 18歳未満
    - 6歳未満
- マンション多い？
    - 共同住宅３階建以上

## 今回作成するエリア

### どの程度の年齢を狙うのが良いかのテスト
- a1: 人口が50％以上、18歳未満が多い（75％）
- a2: 人口が50％以上、6歳未満が多い（75％）
- 旧来のチラシ営業を試す

### どのような営業手法が効率的か？（チラシ、電子広告）
- b1: 人口が50％以上、６歳未満、１８歳未満の両方が多い地域かつ、マンションが多い地域
- 上の地域でデジタル広告とチラシ営業地域を分け、テストする

In [5]:
# 統計量の観察
data.describe()

,KEY_CODE,AREA,JINKO,HYOSYO,HTKSYORI,人口総数,男,女,世帯総数,総数、年齢「不詳」含む,総数０～４歳,総数５～９歳,総数１０～１４歳,総数１５～１９歳,総数２０～２４歳,総数２５～２９歳,総数３０～３４歳,総数３５～３９歳,総数４０～４４歳,総数４５～４９歳,総数５０～５４歳,総数５５～５９歳,総数６０～６４歳,総数６５～６９歳,総数７０～７４歳,総数１５歳未満,総数１５～６４歳,総数６５歳以上,総数７５歳以上,男の総数、年齢「不詳」含む,男０～４歳,男５～９歳,男１０～１４歳,男１５～１９歳,男２０～２４歳,男２５～２９歳,男３０～３４歳,男３５～３９歳,男４０～４４歳,男４５～４９歳,...,Ｃ 鉱業、採石業、砂利採取業,Ｄ 建設業,Ｅ 製造業,Ｆ 電気・ガス・熱供給・水道業,Ｇ 情報通信業,Ｈ 運輸業、郵便業,Ｉ 卸売業、小売業,Ｊ 金融業、保険業,Ｋ 不動産業、物品賃貸業,Ｌ 学術研究、専門・技術サービス業,Ｍ 宿泊業、飲食サービス業,Ｎ 生活関連サービス業、娯楽業,Ｏ 教育、学習支援業,Ｐ 医療、福祉,Ｑ 複合サービス事業,Ｒ サービス業（他に分類されないもの）,Ｓ 公務（他に分類されるものを除く）,Ｔ 分類不能の産業,雇用者（役員を含む）,自営業主（家庭内職者を含む）,家族従業者,総数_y,Ａ 管理的職業従事者,Ｂ 専門的・技術的職業従事者,Ｃ 事務従事者,Ｄ 販売従事者,Ｅ サービス職業従事者,Ｆ 保安職業従事者,Ｇ 農林漁業従事者,Ｈ 生産工程従事者,Ｉ 輸送・機械運転従事者,Ｊ 建設・採掘従事者,Ｋ 運搬・清掃・包装等従事者,Ｌ 分類不能の職業,総数,農林漁業就業者世帯,農林漁業・非農林漁業就業者混合世帯,非農林漁業就業者世帯,非就業者世帯,分類不能の世帯
count,4.777000e+03,4.777000e+03,4777.000000,4777.000000,4777.0,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,...,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000
mean,1.095391e+10,1.604738e+06,774.608541,2.488382,0.0,774.608541,381.200126,393.408415,299.267323,774.215826,30.778522,33.609588,35.632196,35.139418,30.821436,37.457191,42.787105,48.955411,58.255600,52.127277,48.469123,47.457191,53.610634,60.855139,49.525225,100.020306,455.080385,213.687251,103.306887,381.004396,15.769102,17.203684,18.357756,18.103622,15.803642,19.615449,22.128533,25.211848,29.919615,26.698974,...,0.113460,28.404229,97.179401,1.713000,5.299142,20.263345,58.364664,7.831903,5.657944,10.211430,22.981788,13.989533,15.920243,40.635127,3.314214,20.379108,11.279883,11.872514,330.806573,35.328239,14.676366,390.244924,8.789826,54.764915,66.294118,45.606657,44.924011,7.021352,14.537785,76.943060,13.586561,17.321959,29.082269,11.372409,298.691647,4.042077,5.516014,208.093155,72.131463,8.908939
std,1.098564e+10,7.897216e+06,1143.063059,0.499917,0.0,1143.063059,566.388433,578.145549,445.229833,1143.181372,52.546780,55.392786,57.325602,57.419526,55.132478,61.129187,70.424982,77.750847,91.482443,80.902296,71.345908,67.374521,76.004870,87.553857,73.215606,163.176179,688.129804,301.244504,144.565112,566.442383,27.207660,28.477844,29.638100,31.046575,31.085201,33.157271,36.662271,40.279160,46.970952,41.679465,...,0.473879,44.268360,165.151747,5.016504,9.006041,32.275162,86.270941,11.962112,10.022870,16.402116,40.823315,21.946102,25.586228,64.426677,5.243992,29.926622,34.427742,21.535475,498.365029,48.143510,21.164139,574.030566,13.148401,88.375070,97.659608,69.028667,70.308124,33.639332,27.679676,127.475225,21.783244,27.119867,44.170421,20.735353,444.695867,7.336215,9.797949,316.595557,113.544046,16.531753
min,2.210100e+08,8.218120e+02,0.000000,2.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [6]:
# 独自に使いたいデータを作成
# 面積当たり人口
data['per_area'] = data['JINKO'] / data['AREA']
# 面積当たり世帯割合
data['per_setai'] = data['世帯総数'] / data['AREA']
# 子供のいる世帯割合
data['ratio_under18'] = data['\u3000１８歳未満世帯員のいる一般世帯総数'] / data['世帯総数']
data['ratio_under6'] = data['\u3000６歳未満世帯員のいる一般世帯総数'] / data['世帯総数']
# マンションの多い地域
data['apartments'] = (data['共同住宅'] - data['共同住宅１・２階建']) / data['世帯総数']

In [7]:
data.describe()

,KEY_CODE,AREA,JINKO,HYOSYO,HTKSYORI,人口総数,男,女,世帯総数,総数、年齢「不詳」含む,総数０～４歳,総数５～９歳,総数１０～１４歳,総数１５～１９歳,総数２０～２４歳,総数２５～２９歳,総数３０～３４歳,総数３５～３９歳,総数４０～４４歳,総数４５～４９歳,総数５０～５４歳,総数５５～５９歳,総数６０～６４歳,総数６５～６９歳,総数７０～７４歳,総数１５歳未満,総数１５～６４歳,総数６５歳以上,総数７５歳以上,男の総数、年齢「不詳」含む,男０～４歳,男５～９歳,男１０～１４歳,男１５～１９歳,男２０～２４歳,男２５～２９歳,男３０～３４歳,男３５～３９歳,男４０～４４歳,男４５～４９歳,...,Ｈ 運輸業、郵便業,Ｉ 卸売業、小売業,Ｊ 金融業、保険業,Ｋ 不動産業、物品賃貸業,Ｌ 学術研究、専門・技術サービス業,Ｍ 宿泊業、飲食サービス業,Ｎ 生活関連サービス業、娯楽業,Ｏ 教育、学習支援業,Ｐ 医療、福祉,Ｑ 複合サービス事業,Ｒ サービス業（他に分類されないもの）,Ｓ 公務（他に分類されるものを除く）,Ｔ 分類不能の産業,雇用者（役員を含む）,自営業主（家庭内職者を含む）,家族従業者,総数_y,Ａ 管理的職業従事者,Ｂ 専門的・技術的職業従事者,Ｃ 事務従事者,Ｄ 販売従事者,Ｅ サービス職業従事者,Ｆ 保安職業従事者,Ｇ 農林漁業従事者,Ｈ 生産工程従事者,Ｉ 輸送・機械運転従事者,Ｊ 建設・採掘従事者,Ｋ 運搬・清掃・包装等従事者,Ｌ 分類不能の職業,総数,農林漁業就業者世帯,農林漁業・非農林漁業就業者混合世帯,非農林漁業就業者世帯,非就業者世帯,分類不能の世帯,per_area,per_setai,ratio_under18,ratio_under6,apartments
count,4.777000e+03,4.777000e+03,4777.000000,4777.000000,4777.0,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,...,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4777.000000,4651.000000,4651.000000,4651.000000
mean,1.095391e+10,1.604738e+06,774.608541,2.488382,0.0,774.608541,381.200126,393.408415,299.267323,774.215826,30.778522,33.609588,35.632196,35.139418,30.821436,37.457191,42.787105,48.955411,58.255600,52.127277,48.469123,47.457191,53.610634,60.855139,49.525225,100.020306,455.080385,213.687251,103.306887,381.004396,15.769102,17.203684,18.357756,18.103622,15.803642,19.615449,22.128533,25.211848,29.919615,26.698974,...,20.263345,58.364664,7.831903,5.657944,10.211430,22.981788,13.989533,15.920243,40.635127,3.314214,20.379108,11.279883,11.872514,330.806573,35.328239,14.676366,390.244924,8.789826,54.764915,66.294118,45.606657,44.924011,7.021352,14.537785,76.943060,13.586561,17.321959,29.082269,11.372409,298.691647,4.042077,5.516014,208.093155,72.131463,8.908939,0.003451,0.001430,0.214859,0.082288,0.135266
std,1.098564e+10,7.897216e+06,1143.063059,0.499917,0.0,1143.063059,566.388433,578.145549,445.229833,1143.181372,52.546780,55.392786,57.325602,57.419526,55.132478,61.129187,70.424982,77.750847,91.482443,80.902296,71.345908,67.374521,76.004870,87.553857,73.215606,163.176179,688.129804,301.244504,144.565112,566.442383,27.207660,28.477844,29.638100,31.046575,31.085201,33.157271,36.662271,40.279160,46.970952,41.679465,...,32.275162,86.270941,11.962112,10.022870,16.402116,40.823315,21.946102,25.586228,64.426677,5.243992,29.926622,34.427742,21.535475,498.365029,48.143510,21.164139,574.030566,13.148401,88.375070,97.659608,69.028667,70.308124,33.639332,27.679676,127.475225,21.783244,27.119867,44.170421,20.735353,444.695867,7.336215,9.797949,316.595557,113.544046,16.531753,0.004738,0.002166,0.100081,0.052587,0.200178
min,2.210100e+08,8.218120e+02,0.000000,2.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

## 閾値

- per_area 50% (0.002076)
- ratio_under18 75% (0.27) ab要素
- ratio_under6 75% (0.10888) ab要素
- apartments 75% (0.2) ab要素

In [8]:
data_under18 = data.query('per_area > 0.002076 & ratio_under18 > 0.27')
data_under6 = data.query('per_area > 0.002076 & ratio_under6 > 0.10888')

In [9]:
print('---- under18 -----')
print(len(data_under18))
print('---- under6 ----')
print(len(data_under6))

---- under18 -----
602
---- under6 ----
616


In [10]:
# ラベルをつけてデータをマージする
data_under18['label'] = 18
data_under6['label'] = 6
data_under18 = data_under18.to_crs('EPSG:4326')
data_under6 = data_under6.to_crs('EPSG:4326')

merged_data = pd.concat([data_under18, data_under6])

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [11]:
merged_data

,KEY_CODE,AREA,JINKO,HYOSYO,CITYNAME,NAME,HTKSYORI,HTKSAKI,GASSAN,人口総数,男,女,世帯総数,総数、年齢「不詳」含む,総数０～４歳,総数５～９歳,総数１０～１４歳,総数１５～１９歳,総数２０～２４歳,総数２５～２９歳,総数３０～３４歳,総数３５～３９歳,総数４０～４４歳,総数４５～４９歳,総数５０～５４歳,総数５５～５９歳,総数６０～６４歳,総数６５～６９歳,総数７０～７４歳,総数１５歳未満,総数１５～６４歳,総数６５歳以上,総数７５歳以上,男の総数、年齢「不詳」含む,男０～４歳,男５～９歳,男１０～１４歳,男１５～１９歳,男２０～２４歳,男２５～２９歳,...,Ｋ 不動産業、物品賃貸業,Ｌ 学術研究、専門・技術サービス業,Ｍ 宿泊業、飲食サービス業,Ｎ 生活関連サービス業、娯楽業,Ｏ 教育、学習支援業,Ｐ 医療、福祉,Ｑ 複合サービス事業,Ｒ サービス業（他に分類されないもの）,Ｓ 公務（他に分類されるものを除く）,Ｔ 分類不能の産業,雇用者（役員を含む）,自営業主（家庭内職者を含む）,家族従業者,総数_y,Ａ 管理的職業従事者,Ｂ 専門的・技術的職業従事者,Ｃ 事務従事者,Ｄ 販売従事者,Ｅ サービス職業従事者,Ｆ 保安職業従事者,Ｇ 農林漁業従事者,Ｈ 生産工程従事者,Ｉ 輸送・機械運転従事者,Ｊ 建設・採掘従事者,Ｋ 運搬・清掃・包装等従事者,Ｌ 分類不能の職業,総数,農林漁業就業者世帯,農林漁業・非農林漁業就業者混合世帯,非農林漁業就業者世帯,非就業者世帯,分類不能の世帯,WHOLE_NAME,geometry,per_area,per_setai,ratio_under18,ratio_under6,apartments,label
8,221010090,112868.483,1026,2.0,静岡市葵区,緑町,0.0,None,None,1026.0,513.0,513.0,395.0,1026.0,40.0,67.0,74.0,49.0,25.0,39.0,36.0,52.0,97.0,95.0,76.0,76.0,55.0,67.0,42.0,181.0,600.0,243.0,134.0,513.0,23.0,39.0,44.0,32.0,14.0,24.0,...,18.0,24.0,23.0,11.0,41.0,66.0,1.0,22.0,17.0,13.0,420.0,32.0,11.0,471.0,33.0,101.0,138.0,82.0,42.0,8.0,2.0,21.0,5.0,11.0,16.0,12.0,394.0,2.0,1.0,295.0,87.0,9.0,静岡市葵区緑町,"POLYGON ((138.39396 34.98620, 138.39393 34.986...",0.009090,0.003500,0.324051,0.098734,0.339241,18
14,221010160,141898.661,623,2.0,静岡市葵区,瓦場町,0.0,None,None,623.0,301.0,322.0,232.0,623.0,36.0,36.0,23.0,26.0,24.0,26.0,35.0,42.0,40.0,58.0,47.0,36.0,29.0,36.0,44.0,95.0,363.0,165.0,85.0,301.0,19.0,22.0,9.0,13.0,14.0,8.0,...,7.0,10.0,18.0,15.0,23.0,46.0,0.0,19.0,11.0,3.0,252.0,34.0,9.0,298.0,10.0,52.0,80.0,36.0,45.0,6.0,1.0,32.0,8.0,10.0,15.0,3.0,232.0,0.0,0.0,174.0,55.0,3.0,静岡市葵区瓦場町,"POLYGON ((138.39509 34.98081, 138.39488 34.980...",0.004390,0.001635,0.288793,0.142241,0.181034,18
19,221010220,59815.217,297,2.0,静岡市葵区,東町,0.0,None,None,297.0,139.0,158.0,130.0,297.0,16.0,19.0,14.0,11.0,22.0,8.0,17.0,29.0,44.0,27.0,16.0,14.0,13.0,13.0,17.0,49.0,201.0,47.0,17.0,139.0,4.0,9.0,10.0,7.0,12.0,1.0,...,4.0,9.0,4.0,2.0,11.0,12.0,0.0,5.0,9.0,18.0,117.0,9.0,7.0,151.0,2.0,34.0,41.0,34.0,8.0,0.0,0.0,7.0,1.0,3.0,3.0,18.0,130.0,0.0,0.0,87.0,28.0,15.0,静岡市葵区東町,"POLYGON ((138.40032 34.97763, 138.40147 34.977...",0.004965,0.002173,0.292308,0.146154,0.761538,18
67,221010840,12824.487,238,2.0,静岡市葵区,安倍町,0.0,None,None,238.0,108.0,130.0,94.0,238.0,13.0,17.0,7.0,8.0,6.0,5.0,10.0,11.0,21.0,31.0,17.0,24.0,13.0,15.0,9.0,37.0,146.0,55.0,31.0,108.0,5.0,9.0,0.0,3.0,2.0,2.0,...,5.0,3.0,6.0,3.0,12.0,20.0,0.0,4.0,6.0,2.0,105.0,8.0,4.0,119.0,8.0,29.0,36.0,17.0,10.0,1.0,0.0,7.0,1.0,2.0,6.0,2.0,94.0,0.0,0.0,71.0,22.0,1.0,静岡市葵区安倍町,"POLYGON ((138.37594 34.98088, 138.37614 34.980...",0.018558,0.007330,0.308511,0.127660,0.648936,18
74,221010920,151140.873,1196,2.0,静岡市葵区,若松町,0.0,None,None,1196.0,550.0,646.0,440.0,1196.0,59.0,53.0,59.0,55.0,31.0,68.0,59.0,71.0,96.0,85.0,70.0,76.0,64.0,91.0,65.0,171.0,675.0,350.0,194.0,550.0,34.0,24.0,26.0,32.0,15.0,33.0,...,17.0,19.0,28.0,22.0,21.0,79.0,2.0,31.0,16.0,26.0,471.0,50.0,16.0,561.0,12.0,89.0,120.0,77.0,68.0,4.0,2.0,81.0,12.0,25.0,47.0,24.0,439.0,1.0,1.0,312.0,103.0,22.0,静岡市葵区若松町,"POLYGON ((138.36712 34.97826, 138.36704 34.978...",0.007913,0.002911,0.284091,0.125000,0.234091,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4730,22461010039,27835.325,103,3.0,周智郡森町,天宮（本丁）,0.0,None,None,103.0,47.0,56.0,32.0,103.0,4.0,3.0,5.0,3.0,3.0,11.0,5.0,5.0,3.0,7.0,6.0,8.0,8.0,5.0,8.0,12.0,59.0,32.0,19.0,47.0,0.0,1.0,2.0,1.0,0.0,6.0,...,1.0,1.0,0.0,7.0,1.0,6.0,3.0,5.0,2.0,0.0,47.0,5.0,3.0,55.0,0.0,9.0,14.0,0.0,8.0,0.0,0.0,14.0,1.0,3.0,6.0,0.0,32.0,0.0,1.0,22.0,9.0,0.0,周智郡森町天宮（本丁）,"POLYGON ((137.93443 34.84059, 137.93439 34.840...",0.003700,0.001150,0.250000,0.125000,0.000000,6
4740,22461011048,71752.944,189,3.0,周智郡森町,森（下宿）,0.0,None,

In [12]:
data_under18a = data_under18[['KEY_CODE', 'WHOLE_NAME', '人口総数', 'per_area', 'ratio_under18', 'ratio_under6', 'geometry']]
data_under6a = data_under6[['KEY_CODE', 'WHOLE_NAME', '人口総数', 'per_area', 'ratio_under18', 'ratio_under6', 'geometry']]
merged_data_a = pd.concat([data_under18a, data_under6a])

center = shapely.geometry.MultiPolygon(merged_data_a.geometry.values).centroid

map = folium.Map([center.y, center.x])

folium.Choropleth(geo_data=merged_data_a, data=merged_data_a, columns=['KEY_CODE', 'per_area'],
                  key_on='feature.properties.KEY_CODE', fill_color='OrRd', name='total').add_to(map)

folium.GeoJson(
    data_under18a,
    tooltip=folium.features.GeoJsonTooltip(fields=['KEY_CODE', 'WHOLE_NAME', '人口総数', 'ratio_under18', 'per_area']),
    style_function = lambda x: {'color': 'red'},
    name='under18',
).add_to(map)


folium.GeoJson(
    data_under6a,
    tooltip=folium.features.GeoJsonTooltip(fields=['KEY_CODE', 'WHOLE_NAME', '人口総数', 'ratio_under6', 'per_area']),
    style_function = lambda x: {'color': 'blue'},
    name='under6',
).add_to(map)

folium.LayerControl().add_to(map)

map

Output hidden; open in https://colab.research.google.com to view.

In [15]:
# 6, 18歳が多い地域を避ける（対比のため）

data_under18a = data_under18[['KEY_CODE', 'WHOLE_NAME', '人口総数', 'per_area', 'ratio_under18', 'ratio_under6', 'label', 'geometry']]
data_under6a = data_under6[['KEY_CODE', 'WHOLE_NAME', '人口総数', 'per_area', 'ratio_under18', 'ratio_under6', 'label', 'geometry']]

merged_data_a = pd.concat([data_under18a, data_under6a])
dup_df = pd.DataFrame(merged_data_a['KEY_CODE'].value_counts()).reset_index().query('KEY_CODE == 2')
merged_data_a = merged_data_a[~merged_data_a['KEY_CODE'].isin(dup_df['index'])]
merged_data_a['label'].value_counts()

6     189
18    175
Name: label, dtype: int64

In [17]:
merged_data_a['KEY_CODE'].value_counts()

22132011005    1
221340070      1
22211214002    1
221030580      1
222110390      1
              ..
22101127001    1
22101161003    1
222141790      1
22216011101    1
221020160      1
Name: KEY_CODE, Length: 364, dtype: int64

In [18]:
data_under18a = merged_data_a.query('label == 18')
data_under6a = merged_data_a.query('label == 6')

center = shapely.geometry.MultiPolygon(merged_data_a.geometry.values).centroid

map = folium.Map([center.y, center.x])

folium.Choropleth(geo_data=merged_data_a, data=merged_data_a, columns=['KEY_CODE', 'per_area'],
                  key_on='feature.properties.KEY_CODE', fill_color='OrRd', name='total').add_to(map)

folium.GeoJson(
    data_under18a,
    tooltip=folium.features.GeoJsonTooltip(fields=['KEY_CODE', 'WHOLE_NAME', '人口総数', 'ratio_under18', 'per_area']),
    style_function = lambda x: {'color': 'red'},
    name='under18',
).add_to(map)


folium.GeoJson(
    data_under6a,
    tooltip=folium.features.GeoJsonTooltip(fields=['KEY_CODE', 'WHOLE_NAME', '人口総数', 'ratio_under6', 'per_area']),
    style_function = lambda x: {'color': 'blue'},
    name='under6',
).add_to(map)

folium.LayerControl().add_to(map)

map

Output hidden; open in https://colab.research.google.com to view.

In [19]:
# 各地域の人口総数
px.histogram(merged_data_a[['人口総数', 'label']], color='label', barmode='overlay', marginal='violin')

In [20]:
# 各年齢の人口密度
px.histogram(merged_data_a[['per_area', 'label']], color='label', barmode='overlay', marginal='violin')

In [22]:
## ６歳未満、18歳未満ともに多い地域が残るのはもったいない
## そこで、新たな営業手法（デジタル広告）と旧来のチラシ広告の効果をテストする

data_under18b = data_under18[['KEY_CODE', 'WHOLE_NAME', '人口総数', 'per_area', 'ratio_under18', 'ratio_under6', 'label', 'geometry']]
data_under6b = data_under6[['KEY_CODE', 'WHOLE_NAME', '人口総数', 'per_area', 'ratio_under18', 'ratio_under6', 'label', 'geometry']]

merged_data_b = pd.concat([data_under18b, data_under6b])
dup_df = pd.DataFrame(merged_data_b['KEY_CODE'].value_counts()).reset_index().query('KEY_CODE == 2')
merged_data_b = merged_data_b[merged_data_b['KEY_CODE'].isin(dup_df['index'])]

In [24]:
merged_data_b['KEY_CODE'].value_counts()

22132011004    2
22210043201    2
22133021002    2
22133021003    2
221031320      2
              ..
22213134003    2
221031090      2
22213134001    2
221030070      2
22214144004    2
Name: KEY_CODE, Length: 427, dtype: int64

## 要素を考慮しながら、エリアを２分割し、営業手法のテストを

- 広告の出し方、内容などの検討の必要